### **Storing with Artifacts**

This notebook will show how to use Artifact storage. Artifact storage on the eodc ArgoWorkflows relies on S3 storage, the credentials for this needs to be set up for your namespace. Artifacts are not a permanent storage, but can be passed between steps.

**First some imports and global settings**

In [ ]:
from hera.workflows import script, Artifact, DAG, NoneArchiveStrategy, Workflow
from hera.shared import global_config

global_config.host = "https://services.eodc.eu/workflows/"
global_config.namespace = "<YOUR NAMESPACE>"
global_config.token = "<YOUR TOKEN>"
global_config.image = "ghcr.io/eodcgmbh/cluster_image:2025.2.0"

**Writing the scripts**

Contrary to empty directories, no volume has to be set up, however the path to the artifact will be under `/tmp`. To pass artifacts between steps, we need to pass the `output` parameter  in the script decorator. By default, Artifacts will be compressed to tgz format, setting `archive` to `NoneArchiveStrategy` will leave the artifact in it's original format.

In the scripts we will do the same as win the empty directory notebook, but we will pass the downloaded file between steps.

In [ ]:
@script(outputs=Artifact(name="output-artifact", path="/tmp/INCA.nc", archive=NoneArchiveStrategy()))
def create_artifact():
    from urllib.request import urlretrieve
    import datetime
    import os

    artifact_path = "/tmp"

    url = f"https://public.hub.geosphere.at/datahub/resources/inca-v1-1h-1km/filelisting/RR/INCAL_HOURLY_RR_202506.nc"
    urlretrieve(url, os.path.join(artifact_path, "INCA.nc"))

In [ ]:
@script(inputs=Artifact(name="input-artifact", path="/tmp/INCA.nc"))
def consume_artifact():
    import xarray as xr

    artifact_path="/tmp/INCA.nc"

    ds = xr.open_dataset(os.path.join(artifact_path, "INCA.nc")).isel(time=1, x=1, y=1)["RR"].load()
    print(ds.values)

**Creating the Workflow**

When creating the workflow we need to pass the artifact created in the create step to the consume step with `get_artifact`, here the name of the artifact needs to be specified. It is also possible to pass other parameter required in the function as a dictionary in the list of arguments. 

In [ ]:
with Workflow(
    generate_name="artifact-storage-",
    entrypoint="workflow"
) as w:
    with DAG(name="workflow"):
        create = create_artifact()
        consume = consume_artifact(arguments=[create.get_artifact("output-artifact").with_name("input-artifact")])

        create >> consume

**Submitting the Workflow**

In [ ]:
w.create()